In [ ]:
# Create a joint dataframe and remove the duplicates in the shop id
# Note that some shop names are the same despite the spelling differences
df = pd.merge(items, items_category, left_on = "item_category_id", right_on = "item_category_id")
df = pd.merge(sales, df, left_on = "item_id", right_on = "item_id")
df = pd.merge(df, shops, left_on = "shop_id", right_on = "shop_id")
    
# convert to datetime and sort the values
df.sort_values(by = ["shop_id", "date"], ascending = True, inplace = True)

# df.head()

# Note that some shop names contain punctuations and should be corrected, some are the same despite the spelling error
d = {0:57, 1:58, 10:11, 23:24}
df["shop_id"] = df["shop_id"].apply(lambda x: d[x] if x in d.keys() else x)


# Calcultae the monthly sales
df["date"] = pd.to_datetime(df["date"], format = "%d.%m.%Y")
sales_month=df[["date", "item_cnt_day"]].set_index("date").resample("M").sum()
sales_week=df[["date","item_cnt_day"]].set_index("date").resample("W").sum()

fig,axs = plt.subplots(2,1,figsize=(16,9))
axs[0].plot(sales_month,color="red",label="Monthly sales")
axs[0].set_title('Monthly Sales')
axs[1].plot(sales_week,color="blue",label="Weekly sales")
axs[1].set_title('Weekly Sales')
plt.xlabel("Date")
plt.ylabel("Sales")
plt.show()

# Note that the first word in the shop name is the city name, but first we need to clean the city names
df["shop_name"]=df["shop_name"].apply(lambda s: "".join([x for x in s if x not in string.punctuation]))
df["city"]=df["shop_name"].apply(lambda s: s.split()[0])
df["city"].head()

# Create City treep, find the most important cities
df_city=df[["city","item_cnt_day"]].groupby("city")["item_cnt_day"].sum().to_frame()
values=df_city["item_cnt_day"]
labels = ["{} - Sales :{}k".format(city, sales/1000) for city, sales in zip(df_city.index, values)]

plt.figure(figsize = (30, 8))
squarify.plot(sizes = values, label = labels,  alpha = 0.8)
plt.title("Sales by city",fontsize = 23, fontweight = "bold")

from statsmodels.graphics.tsaplots import plot_acf

plt.figure()
plot_acf(df.set_index("date").resample("D")["item_cnt_day"].sum(), lags = 14)
plt.show()

In [ ]:
best_aic = np.inf 
best_order = None
best_mdl = None


# model=ARIMA(np.linspace(1,10),order=(1,1,0)).fit()
rng = range(5)
for i in rng:
    for j in rng:
        try:
            tmp_mdl = ARIMA(sales_month.values, order=(i, j,0)).fit()
            tmp_aic = tmp_mdl.aic
#             print(tmp_aic)
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (i, j)
                best_mdl = tmp_mdl
        except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))

In [ ]:
def restored_diff(sales_shop,sales_shop_diff,periods):
    NumShops=sales_shop.shape[1]
    NumPeriods=sales_shop.shape[0]
    restored = sales_shop.copy()
    restored.iloc[periods:,:] = np.nan

    for k in range(NumShops):
        for d in range(periods,NumPeriods):
            restored.iloc[d,k] = restored.iloc[d-periods,k] + sales_shop_diff.iloc[d,k]
            
    return restored